<a href="https://colab.research.google.com/github/RPGNZ/Pivot-point-DCA-strategy/blob/main/Backtest_pivot_DCA_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pivot point based DCA strategy vs. regular DCA

### Disclaimer
The Python code presented here is for educational and informational purposes only. It is not intended as investment advice or as a recommendation to buy, sell or hold any particular asset. The author is not a financial advisor and is not responsible for any investment decisions made by the reader. The reader should conduct their own research and analysis before making any investment decisions. The author is not liable for any losses or damages that may arise from the use of this code.

### Imports

In [91]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import math
!pip install numpy-financial
import numpy_financial as npf

### Functions

#### Weekly pivot-based DCA

In [92]:
def pivot_levels_for_display(weekly_pivot_levels):
    # Reshape weekly_pivot_level for display

    # Keep only pivot columns
    pivot_points_display = weekly_pivot_levels.loc[:, ["S3", "S2", "S1", "P", "R1", "R2", "R3"]].copy()

    # Add start date of the week (Monday-like window)
    pivot_points_display["week_start"] = pivot_points_display.index - pd.Timedelta(days=6)

    # Rename index for clarity
    pivot_points_display = pivot_points_display.rename_axis("week_end").reset_index()

    pivot_points_display = pivot_points_display.melt(
        id_vars=["week_start", "week_end"],
        value_vars=["S3", "S2", "S1", "P", "R1", "R2", "R3"],
        var_name="pivot_name",
        value_name="pivot_value"
    )

    return pivot_points_display

def allocate_round_robin(pivots: dict, cash: float, max_pivots: int = 2):
    # Cash is allocated to pivot point prices in a round-robin fashion across a limited number of pivot levels (default: 2).
    #
    # Parameters
    # ----------
    # pivots:     dict {pivot_name: price}
    # cash:       cash to be allocated
    # max_pivots: max number of pivots to use

    # Initialize allocations to 0
    allocations = {pivot: 0 for pivot in pivots}

    # Select only first pivot levels
    selected_pivots = list(pivots.items())[:max_pivots]

    # round-robin loop
    while True:
        bought_any = False

        for pivot, price in selected_pivots:
            if cash >= price:
                allocations[pivot] += 1
                cash -= price
                bought_any = True

        # No additionnal allocation → stop
        if not bought_any:
            break

    return allocations


def weekly_pivot_DCA(daily_data: pd.DataFrame, cash: float, allow_fractional_shares: bool, pivot_DCA_param):
    # Open and execute limit orders on pivot levels bellow last week close price when price is bellow EMA.
    # Otherwise buy ate the open level of the first day of the week (like regular DCA)

    data = daily_data.copy() # creates a copy of daily_data dataframe

    # Compute daily EMA to be used as strategy arbitration
    data['EMA'] = data['Close'].ewm(span=pivot_DCA_param["ema_period"], adjust=False).mean()

    data['Total_cash_invested'] = float('nan') # Total amount of cash put in the DCA strategy
    data['Order_size'] = float('nan')          # Size of current week order

    # Weekly pivot points
    weekly = data.resample('W-SUN').agg({'High': 'max', 'Low': 'min', 'Close': 'last'}) # resample data to Sundays, index of the week is on Sundays
    weekly['P'] = (weekly['High'] + weekly['Low'] + weekly['Close']) / 3
    weekly['R1'] = 2 * weekly['P'] - weekly['Low']
    weekly['S1'] = 2 * weekly['P'] - weekly['High']
    weekly['R2'] = weekly['P'] + (weekly['High'] - weekly['Low'])
    weekly['S2'] = weekly['P'] - (weekly['High'] - weekly['Low'])
    weekly['R3'] = weekly['High'] + 2 * (weekly['P'] - weekly['Low'])
    weekly['S3'] = weekly['Low']  - 2 * (weekly['High'] - weekly['P'])
    weekly_pivot_levels = weekly.shift(1) # Pivot points of week n are applied on week n+1

    pivot_points_display = pivot_levels_for_display(weekly_pivot_levels) # reshape for display


    executed_orders = [] # All executed orders
    cash_pivot = 0 # current amount of cash available for the strategy
    total_cash_invested = 0 # Cumulative amount of cash allocated to the strategy

    for week_end in weekly_pivot_levels.index[1:]:  # strategy starts at the second week since pivot point haven't been computed before

        cash_pivot += cash
        total_cash_invested += cash

        # get daily data of the week
        week_data = data.loc[week_end - pd.Timedelta(days=6) : week_end]

        if week_data.empty:
            continue

        week_start = week_data.index[0]

        prev_week_end_date = week_start - pd.Timedelta(days=1)

        # Close price of previous week
        close_prev_week = data.loc[data.index < week_start, 'Close'].iloc[-1]
        ema_prev_week = data.loc[data.index < week_start, 'EMA'].iloc[-1]

        # Buy at open price of the week (like DCA) when price is above EMA
        if close_prev_week > ema_prev_week and pivot_DCA_param["ema_filter"]:
            open_price = data.loc[week_start, 'Open']
            order_size = cash_pivot / open_price if allow_fractional_shares else np.floor(cash_pivot / open_price)
            cash_pivot -= order_size * open_price

            executed_orders.append({
                'pivot': 'DCA_OPEN',
                'price': open_price,
                'invested': order_size * open_price,
                'exec_day': week_start,
                'order_size': order_size
            })

            data.at[week_start, 'Order_size'] = order_size
            data.at[week_start, 'Invested'] = order_size*open_price
            data.at[week_start, 'Available_cash'] = cash_pivot
            data.at[week_start, 'Total_cash_invested'] = total_cash_invested


        else: # Use pivot levels to set limit orders

            # Get pivot levels of the week
            piv = weekly_pivot_levels.loc[week_end]
            pivot_levels = {
                'R3': piv['R3'],
                'R2': piv['R2'],
                'R1': piv['R1'],
                'P' : piv['P'],
                'S1': piv['S1'],
                'S2': piv['S2'],
                'S3': piv['S3']
            }

            # Select pivot levels to use for limit orders (below close level of previous week)
            eligible_pivots = {k: v for k, v in pivot_levels.items() if v < close_prev_week} # All pivots below close
            sorted_pivots = sorted(eligible_pivots.items(), key=lambda x: x[1], reverse=True) # Sort by price descending
            pivots_to_trade = dict(sorted_pivots[:pivot_DCA_param["max_pivots"]]) # Keep only the max_pivots closest levels

            if len(pivots_to_trade) == 0:
                continue

            # Begining of the week: add weekly cash
            print(f"\nDate : {week_start}") # first day of the week
            data.at[week_start, 'Order_size'] = 0
            data.at[week_start, 'Invested'] = 0
            data.at[week_start, 'Available_cash'] = cash_pivot
            data.at[week_start, 'Total_cash_invested'] = total_cash_invested

            # allocate cash on the available orders
            size_by_pivot = pivots_to_trade.copy() # init to get the right size

            if allow_fractional_shares:
                size_by_pivot = {k: cash_pivot / len(pivots_to_trade) / v for k, v in pivots_to_trade.items()}
            else:
                allocations = allocate_round_robin(pivots_to_trade, cash_pivot, pivot_DCA_param["max_pivots"])
                for pivot_level, size in allocations.items():
                    size_by_pivot[pivot_level] = size

            print(f"size_by_pivot : {size_by_pivot}")

            # Loop on each pivot level of the week
            for name, price in pivots_to_trade.items():
                # Loop over each day of the week to check if the order is executed
                for idx, row in week_data.iterrows():
                    if row['Low'] <= pivots_to_trade[name]:  # order is executed
                        if row['Open'] <= pivots_to_trade[name]:  # in case open is below pivot level
                            order_price = row['Open']
                        else :
                            order_price = price

                        order_size = size_by_pivot[name]
                        cash_pivot -= order_size * order_price
                        print(f"Execution {name} on {idx} {order_size * order_price} --> {order_size} @ {order_price}")

                        executed_orders.append({
                            'pivot': name,
                            'price': order_price,
                            'invested': order_size * order_price,
                            'exec_day': idx,
                            'order_size': order_size
                            })

                        data.at[idx, 'Order_size'] = data.at[idx, 'Order_size'] + order_size if not pd.isna(data.at[idx, 'Order_size']) else order_size
                        data.at[idx, 'Invested'] = data.at[idx, 'Invested'] + order_size * order_price if not pd.isna(data.at[idx, 'Invested']) else order_size * order_price
                        data.at[idx, 'Available_cash'] = cash_pivot
                        break

        data.at[week_start, 'Total_cash_invested'] = total_cash_invested

    data['Total_cash_invested'] = data['Total_cash_invested'].ffill().fillna(0)

    data['Total_invested'] = data['Invested'].cumsum().ffill().fillna(0)
    data['Position'] = data['Order_size'].cumsum().ffill().fillna(0)
    data['Available_cash'] = data['Available_cash'].ffill().fillna(0)
    data['Portfolio_value'] = data['Position'] * data['Close'] + data['Available_cash']
    data['Average_buy_price'] = data['Total_invested'] / data['Position']

    executed_orders_df = pd.DataFrame(executed_orders)

    return data, pivot_points_display, executed_orders_df

#### Weekly DCA

In [93]:
def weekly_DCA(daily_data: pd.DataFrame, cash: float, allow_fractional_shares: bool):
    # Buy 'cash' amount of the asset at the open level every first day of the week
    # (mostly Mondays but some Mondays are missing in daily_data because market can be closed)

    dca_history = []
    order_size = 0          # this week order size
    position_size = 0       # cumulative position size
    total_cash_invested = 0 # total amount of cash allocated to the strategy
    cash_dca = 0            # current amount of cash available

    data = daily_data.copy() # creates a copy of daily_data dataframe

    data['Order_size'] = float('nan')          # Size of current week order
    data['Invested'] = 0.0                     # Amount spent to buy assets
    data['Available_cash'] = float('nan')      # Remaing cash after the order is executed
    data['Total_cash_invested'] = float('nan') # Total amount of cash put in the DCA strategy

    prev_day_week = data.index[0].isocalendar().week

    for date, row in data.iloc[1:].iterrows():
        if date.isocalendar().week != prev_day_week: # test whether we've started a new week
            prev_day_week = date.isocalendar().week
            total_cash_invested += cash

            open_price = row['Open']
            if allow_fractional_shares:
                order_size = cash / open_price
            else:
                cash_dca += cash
                order_size = np.floor(cash_dca / open_price)
                cash_dca -= order_size * open_price

            data.at[date, 'Order_size'] = order_size
            data.at[date, 'Invested'] = order_size*open_price
            data.at[date, 'Available_cash'] = cash_dca
            data.at[date, 'Total_cash_invested'] = total_cash_invested

    data['Total_invested'] = data['Invested'].cumsum()
    data['Position'] = data['Order_size'].cumsum().ffill().fillna(0)
    data['Available_cash'] = data['Available_cash'].ffill().fillna(0)
    data['Portfolio_value'] = data['Position'] * data['Close'] + data['Available_cash']
    data['Total_cash_invested'] = data['Total_cash_invested'].ffill().fillna(0)
    data['Average_buy_price'] = data['Total_invested'] / data['Position']

    return data

#### Display

In [94]:
def plot_pivot_dca_backtest(
    data: pd.DataFrame,
    df_pivot_dca: pd.DataFrame,
    df_dca: pd.DataFrame,
    executed_orders: pd.DataFrame,
    pivot_points_display: pd.DataFrame,
    pivot_DCA_param: dict,
    ticker: str
):
    # Plot comparison between Pivot-DCA strategy and regular DCA strategy.
    #
    # Parameters
    # ----------
    # data:                 Raw OHLC price data
    # df_pivot_dca:         Strategy dataframe for Pivot-DCA
    # df_dca:               Strategy dataframe for regular DCA
    # executed_orders:      Executed orders dataframe
    # pivot_points_display: Weekly pivot segments with columns:
    #                           ['week_start', 'week_end', 'pivot_name', 'pivot_value']
    # pivot_DCA_param:      Strategy parameters (colors, EMA period...)
    # ticker:               Asset ticker symbol

    fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01,
                    # row_heights=[0.7, 0.1, 0.1, 0.1],
                    row_heights=[0.4, 0.2, 0.2, 0.2],
                    )


    ## Top part
    # ----------
    # - Asset candle bars
    # - EMA
    # - Pivot levels
    # - pivot-based strategy executed orders

    # Candle bars
    fig.add_trace(go.Candlestick(
        x=data.index,
        open=data['Open'],
        high=data['High'],
        low=data['Low'],
        close=data['Close'],
        name=f"{ticker}",
        increasing_line_color='green',
        decreasing_line_color='red',
        opacity=0.9
    ), row=1, col=1)


    # EMA
    fig.add_trace(
        go.Scatter(
            x = df_pivot_dca_fun.index,
            y = df_pivot_dca_fun['EMA'],
            mode='lines',
            name=f'EMA {pivot_DCA_param["ema_period"]}',
            line=dict(color='yellow', width=2)
        ),
        row=1, col=1
    )


    # Weekly pivot levels
    for row in weelky_pivot_levels_display.itertuples(index=False):
        fig.add_trace(
            go.Scatter(
                x=[row.week_start, row.week_end],
                y=[row.pivot_value, row.pivot_value],
                mode="lines",
                line=dict(color=pivot_DCA_param["pivot_colors"][row.pivot_name], width=1, dash="dot"),
                name=row.pivot_name,
                showlegend=False
            ),
            row=1,
            col=1
        )


    # Executed orders
    if not executed_orders.empty:
        exec_orders = executed_orders[executed_orders['order_size'] > 0].copy()
        fig.add_trace(
            go.Scatter(
                x = exec_orders['exec_day'],
                y = exec_orders['price'],
                mode = 'markers',#'markers+text',
                name = 'Executed orders',
                marker = dict(size=10, color='lime', symbol='triangle-up'),
                text = exec_orders['pivot'],
                textposition = 'top center',
                customdata = exec_orders[['order_size', 'invested']].values,
                hovertemplate = '<br>Date: %{x}<br>Level: %{text}<br>Price: %{y:.2f}<br>Size: %{customdata[0]:.4f}<br>Invested: %{customdata[1]:.2f}<extra></extra>'
            ),
            row=1, col=1
        )

    ## Comparison part
    # ----------
    # - Positions
    # - Portfolio value
    # - Average entry price

    # Pivot DCA
    fig.add_trace( go.Scatter(
        x=df_pivot_dca_fun.index,
        y = df_pivot_dca_fun['Position'],
        # y = df_pivot_dca_fun['Portfolio_value'],
        # y = df_pivot_dca_fun['Available_cash'],
        # y= df_pivot_dca_fun['Average_buy_price'],
        mode='lines',
        name='Pivot-DCA (Weekly)',
        line=dict(color='orange', width=2)
    ),row=2, col=1)

    fig.add_trace( go.Scatter(
        x=df_pivot_dca_fun.index,
        # y = df_pivot_dca_fun['Position'],
        y = df_pivot_dca_fun['Portfolio_value'],
        # y = df_pivot_dca_fun['Available_cash'],
        # y= df_pivot_dca_fun['Average_buy_price'],
        mode='lines',
        name='Pivot-DCA (Weekly)',
        showlegend=False,
        line=dict(color='orange', width=2)
    ),row=3, col=1)

    fig.add_trace( go.Scatter(
        x=df_pivot_dca_fun.index,
        # y = df_pivot_dca_fun['Position'],
        # y = df_pivot_dca_fun['Portfolio_value'],
        # y = df_pivot_dca_fun['Available_cash'],
        y= df_pivot_dca_fun['Average_buy_price'],
        mode='lines',
        name='Pivot-DCA (Weekly)',
        showlegend=False,
        line=dict(color='orange', width=2)
    ),row=4, col=1)


    # DCA
    fig.add_trace(go.Scatter(
        x=df_dca_fun.index,
        y = df_dca_fun['Position'],
        # y = df_dca_fun['Portfolio_value'],
        # y = df_dca_fun['Available_cash'],
        # y = df_dca_fun['Average_buy_price'],
        mode='lines',
        name='DCA (Weekly)',
        line=dict(color='cyan', width=2)
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=df_dca_fun.index,
        # y = df_dca_fun['Position'],
        y = df_dca_fun['Portfolio_value'],
        # y = df_dca_fun['Available_cash'],
        # y = df_dca_fun['Average_buy_price'],
        mode='lines',
        name='DCA (Weekly)',
        showlegend=False,
        line=dict(color='cyan', width=2)
    ), row=3, col=1)

    fig.add_trace(go.Scatter(
        x=df_dca_fun.index,
        # y = df_dca_fun['Position'],
        # y = df_dca_fun['Portfolio_value'],
        # y = df_dca_fun['Available_cash'],
        y = df_dca_fun['Average_buy_price'],
        mode='lines',
        name='DCA (Weekly)',
        showlegend=False,
        line=dict(color='cyan', width=2)
    ), row=4, col=1)


    # Layout
    fig.update_yaxes(row=1, col=1, range=[min(data['Low'])*0.99, max(data['High'])*1.01])
    fig.update_yaxes(title_text="Position", row=2, col=1)
    fig.update_yaxes(title_text="Portfolio val.", row=3, col=1)
    fig.update_yaxes(title_text="Av. entry price", row=4, col=1)

    fig.update_layout(
        template='plotly_dark',
        title=f"📊 Pivot point based DCA vs. DCA (Weekly) — {ticker}",
        height=750,
        # width=800,
        xaxis_rangeslider_visible=False,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    fig.show()

#### Misc

In [95]:
def weekly_dca_irr(weekly_budget: float,
                   n_weeks: int,
                   final_value: float):
    # Computes the IRR of a weekly DCA strategy with constant contributions.
    #
    # Parameters
    # ----------
    # weekly_budget: Amount invested every week (negative cashflow).
    # n_weeks:       Number of weeks
    # final_value:   Final portfolio value at the end (positive cashflow).
    #
    # Returns
    # -------
    # weekly_irr:     Weekly internal rate of return.
    # annualized_irr: Annualized IRR equivalent (compounded).

    # Cashflows: weekly investments + final portfolio liquidation
    cashflows = [-weekly_budget] * n_weeks
    cashflows.append(final_value)

    # Weekly IRR
    weekly_irr = npf.irr(cashflows)

    # Annualize (52 weeks per year)
    annualized_irr = (1 + weekly_irr)**52 - 1

    return weekly_irr, annualized_irr

### Parameters

In [96]:
# ticker = "QQQ" # NASDAQ-100
# ticker = "SPY" # S&P500
# ticker = "IWM" # Russel 2000
# ticker = "URTH" # MSCI World
# ticker = "GLD" # Gold
# ticker = "EXS1" # DAX
# ticker = "FXI" # iShares China Large-Cap
# ticker = "USO" # United States Oil Fund
ticker = "BTC-USD"
# ticker = "AAPL"
# ticker = "TTE.PA" # TotalEnergies
# ticker = "RNL.F" # Renault

start_date = "2025-02-25" #"2025-10-26"
weekly_budget = 1000  # weekly budget to be invested
allow_fractional_shares = True # buy fractional shares of the asset

In [97]:
# pivot-based DCA parameters
pivot_DCA_param = {
    "max_pivots": 2, # Depth of pivot levels to use
    "ema_filter": True, # Use EMA position to arbitrate the strategy
    "ema_period": 14, # EMA period
    "pivot_colors": { # for display
        "S3": "red",
        "S2": "red",
        "S1": "orange",
        "P": "white",
        "R1": "cyan",
        "R2": "lime",
        "R3": "lime",
        }
}

### Backtest

In [98]:
# =====================
# 1. download daily data
# =====================
# Download data starting at Monday before start_date for accurate pivot level computation of first week
start_date = pd.to_datetime(start_date)
start_monday = start_date - pd.Timedelta(days=start_date.weekday())

data = yf.download(ticker, start=start_monday, interval="1d", auto_adjust=True)

# if the columns have a MultiIndex, flatten them.
if isinstance(data.columns, pd.MultiIndex):
    data.columns = [col[0] for col in data.columns]


# =====================
# 2. Execute and compare strategies
# =====================

# execute pivot-based strategy
[df_pivot_dca_fun, weelky_pivot_levels_display, executed_orders] = weekly_pivot_DCA(data, weekly_budget, allow_fractional_shares, pivot_DCA_param)

total_cash_invested_pivot = df_pivot_dca_fun['Total_cash_invested'].iloc[-1]
final_portfolio_value_pivot = df_pivot_dca_fun['Portfolio_value'].iloc[-1]

perf_pivot = final_portfolio_value_pivot / total_cash_invested_pivot - 1

w_irr, ann_irr_pivot = weekly_dca_irr(weekly_budget, int(total_cash_invested_pivot/weekly_budget), final_portfolio_value_pivot)


# execute DCA strategy
df_dca_fun = weekly_DCA(data, weekly_budget, allow_fractional_shares)

total_cash_invested_dca = df_dca_fun['Total_cash_invested'].iloc[-1]
final_portfolio_value_dca = df_dca_fun['Portfolio_value'].iloc[-1]

perf_dca = final_portfolio_value_dca / total_cash_invested_dca - 1

w_irr, ann_irr_dca = weekly_dca_irr(weekly_budget, int(total_cash_invested_dca/weekly_budget), final_portfolio_value_dca)

[*********************100%***********************]  1 of 1 completed


Date : 2025-03-10 00:00:00
size_by_pivot : {'S1': np.float64(0.0066115033132929015), 'S2': np.float64(0.007077091532116228)}

Date : 2025-03-17 00:00:00
size_by_pivot : {'P': np.float64(0.012271583566368255), 'S1': np.float64(0.012867551643700714)}
Execution P on 2025-03-18 00:00:00 999.9999999999999 --> 0.012271583566368255 @ 81489.07552083333

Date : 2025-03-31 00:00:00
size_by_pivot : {'S1': np.float64(0.006274658878333481), 'S2': np.float64(0.006490413844674728)}
Execution S1 on 2025-04-06 00:00:00 500.0 --> 0.006274658878333481 @ 79685.60677083334

Date : 2025-04-07 00:00:00
size_by_pivot : {'S1': np.float64(0.010127949765369165), 'S2': np.float64(0.01073106977811054)}

Date : 2025-06-02 00:00:00
size_by_pivot : {'S1': np.float64(0.0048886651079803), 'S2': np.float64(0.005055472887749044)}
Execution S1 on 2025-06-05 00:00:00 500.0 --> 0.0048886651079803 @ 102277.40885416666

Date : 2025-06-16 00:00:00
size_by_pivot : {'S1': np.float64(0.004898972221143484), 'S2': np.float64(0.005

### Results

In [99]:
print("\n=== 📊 Pivot-Based DCA vs DCA ===")
print(f"\n--- Pivot-based DCA ---")
print(f"Total cash invested: {df_pivot_dca_fun['Total_cash_invested'].iloc[-1]:,.0f}")
print(f"Final portfolio value: {final_portfolio_value_pivot:,.0f}")
print(f"Total return: {perf_pivot:.2%}")
print(f"Annualized IRR: {ann_irr_pivot:.2%}")
print(f"{ticker} final position: {df_pivot_dca_fun['Position'].iloc[-1]:.4f}")
print(f"Remaining cash : {df_pivot_dca_fun['Available_cash'].iloc[-1]:.2f}")


print(f"\n--- DCA ---")
print(f"Total cash invested: {df_dca_fun['Total_cash_invested'].iloc[-1]:,.0f}")
print(f"Final portfolio value: {df_dca_fun['Portfolio_value'].iloc[-1]:,.0f}")
print(f"Total return: {perf_dca:.2%}")
print(f"Annualized IRR: {ann_irr_dca:.2%}")
print(f"{ticker} final position: {df_dca_fun['Position'].iloc[-1]:.4f}")
print(f"Remaining cash : {df_dca_fun['Available_cash'].iloc[-1]:.2f}")

plot_pivot_dca_backtest(
    data=data,
    df_pivot_dca=df_pivot_dca_fun,
    df_dca=df_dca_fun,
    executed_orders=executed_orders,
    pivot_points_display = weelky_pivot_levels_display,
    pivot_DCA_param=pivot_DCA_param,
    ticker=ticker
)


=== 📊 Pivot-Based DCA vs DCA ===

--- Pivot-based DCA ---
Total cash invested: 52,000
Final portfolio value: 37,261
Total return: -28.34%
Annualized IRR: -50.00%
BTC-USD final position: 0.5419
Remaining cash : 0.00

--- DCA ---
Total cash invested: 52,000
Final portfolio value: 36,995
Total return: -28.86%
Annualized IRR: -50.79%
BTC-USD final position: 0.5380
Remaining cash : 0.00
